In [ ]:
import pandas as pd
import datetime
#import dask.dataframe as dd
from sklearn import preprocessing
import numpy as np

SYMBOL_FACTOR = 100

folder = "data/GJ_short5/"

In [ ]:
pd.options.mode.chained_assignment = None # Turn off warnings on creating new Year/Month/Time columns

startTime = '2019-01-03 00:00:00'
endTime = '2020-05-28 23:59:59'

SYMBOL = 'GBPJPY'
def prepareDF_signal(dataframe,getVolume, tf_string):
    dataframe['Time']=pd.to_datetime(dataframe['Time'], format='%Y.%m.%d %H:%M')
    #if not getVolume:
    dataframe['Time'] = dataframe['Time'].shift(-1)
    timeSeries_Start = pd.to_datetime(startTime)
    timeSeries_End = pd.to_datetime(endTime)
    dataframe1 = dataframe.loc[dataframe['Time'] >= timeSeries_Start]
    dataframe3 = dataframe1.loc[dataframe1['Time'] <= timeSeries_End]
    
    if getVolume:
        dataframe3['Year'] = pd.DatetimeIndex(dataframe3['Time']).year
        dataframe3['Month'] = pd.DatetimeIndex(dataframe3['Time']).month
        dataframe3['Day'] = pd.DatetimeIndex(dataframe3['Time']).day
        dataframe3 = dataframe3[['Time', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Open', 'Close', 'High', 'Low', 'Volume', 'RSI', 'MA']]     
        return dataframe3
    
    elif tf_string == '1440_':
        dataframe4 = dataframe3.drop(['Hour','Minute','DayOfWeak', 'Open'], axis=1)
        dataframe4.rename(columns={'High': tf_string+'High', 'Low': tf_string+'Low', 'Close': tf_string+'Close'}, inplace=True)
        dataframe4 = dataframe4.iloc[:,:4]
        return dataframe4
    
    else:
        dataframe4 = dataframe3.drop(['Hour','Minute','DayOfWeak', 'Open', 'High', 'Low', 'Close'], axis=1)
        dataframe4.rename(columns={'MA': tf_string+'MA', 'MA50': tf_string+'MA50','Volume': tf_string+'Volume', 'High': tf_string+'High', 'Low': tf_string+'Low', 'Close': tf_string+'Close',
                                  'MACD_0': tf_string+'MACD_0', 'MACD_1': tf_string+'MACD_1', 'Momentum': tf_string+'Mom', 'RSI': tf_string+'RSI', 'ATR': tf_string+'ATR', 'StdDev': tf_string+'StdDev', 'Stochastic_0': tf_string+'stoc_0', 'Stochastic_1': tf_string+'stoc_1'}, inplace=True)
    
        return dataframe4

In [ ]:
def mergeDF(lowTF,highTF):
    tf_merge = pd.merge(lowTF, highTF, on='Time', how='outer')
    fill_merge = tf_merge.fillna(method='ffill')
    return fill_merge

In [ ]:
def trend(row):
    if row.iloc[3] > row.iloc[4]:
        return 1
    else:
        return 0

LOAD CSV FILES WITH SIGNALS

In [ ]:
day_csv = pd.read_csv(f'{folder}{SYMBOL}_1440.csv')
day = prepareDF_signal(day_csv,False, '1440_')

In [ ]:
oneHour_csv = pd.read_csv(f'{folder}{SYMBOL}_60.csv')
oneHour = prepareDF_signal(oneHour_csv,False, '60_')

In [ ]:
fifteen_csv = pd.read_csv(f'{folder}{SYMBOL}_15.csv')
fifteen = prepareDF_signal(fifteen_csv,False, '15_')
fifteen['Trend'] = fifteen.apply(lambda row: trend(row), axis=1)

In [ ]:
five_csv = pd.read_csv(f'{folder}{SYMBOL}_5.csv')
five = prepareDF_signal(five_csv,True, '5_')

MERGE SIGNAL DATAFRAMES

In [ ]:
mergefifteen = mergeDF(five,fifteen)
mergeHour = mergeDF(mergefifteen,oneHour)
mergeDay = mergeDF(mergeHour,day)

In [ ]:
def pivot(df):
    data = df.to_numpy()
    h_l_c = data[:,-3:]
    high = data[:,-3:-2]
    low = data[:,-2:-1]
    close = data[:,-1]
    five_close = data[:,7:8]
       
    p = np.expand_dims(np.sum(h_l_c, axis=1)/3, axis=1)
    r1 = np.subtract((2 * p), low)
    r2 = np.subtract(np.add(p, high), low)
    r3 = np.add(high, 2*(np.subtract(p, low)))
    s1 = np.subtract((2 * p), high)
    s2 = np.add(np.subtract(p, high), low)
    s3 = np.subtract(low, 2*np.subtract(high, p))
   
    p_dist = np.subtract(five_close, p)*SYMBOL_FACTOR
    r1_dist = np.subtract(five_close, r1)*SYMBOL_FACTOR
    r2_dist = np.subtract(five_close, r2)*SYMBOL_FACTOR
    r3_dist = np.subtract(five_close, r3)*SYMBOL_FACTOR
    s1_dist = np.subtract(five_close, s1)*SYMBOL_FACTOR
    s2_dist = np.subtract(five_close, s2)*SYMBOL_FACTOR
    s3_dist = np.subtract(five_close, s3)*SYMBOL_FACTOR

    return np.hstack((p_dist,r1_dist,r2_dist,r3_dist,s1_dist,s2_dist,s3_dist)) #, p, r1, r2, r3, s1, s2, s3
   

In [ ]:
new_df = mergeDay.drop(['Time', 'Year', 'MA', '15_MA', '60_MA', '15_MA50', '60_MA50'], axis=1)
new_df['Close_diff'] = (mergeDay['Close'].diff(periods=1))*SYMBOL_FACTOR
new_df['high_low_diff'] = ((mergeDay.iloc[:,8:10].diff(axis=1, periods=-1)).iloc[:,:1])*SYMBOL_FACTOR

In [ ]:
new_df['MA_trend'] = (mergeDay['MA'].diff(periods=1))*SYMBOL_FACTOR
new_df['15_MA_trend'] = (mergeDay['15_MA'].diff(periods=1))*SYMBOL_FACTOR
new_df['15_MA_dist'] = (mergeDay['Close'] - mergeDay['15_MA'])
new_df['60_MA_dist'] = (mergeDay['Close'] - mergeDay['60_MA'])
new_df['15_MA50_dist'] = (mergeDay['Close'] - mergeDay['15_MA50'])
new_df['60_MA50_dist'] = (mergeDay['Close'] - mergeDay['60_MA50'])

In [ ]:
pivot_df = pd.DataFrame(pivot(mergeDay), columns=['p_dist','r1_dist','r2_dist','r3_dist','s1_dist','s2_dist','s3_dist']) #, 'p', 'r1', 'r2', 'r3', 's1', 's2', 's3'
data_final = pd.concat([new_df, pivot_df], axis=1).iloc[1:,:]

In [ ]:
data_clean = data_final.drop(['Open', '1440_High',
       '1440_Low', '1440_Close'], axis=1)

In [ ]:
#USED FOR XGBOOST MODEL

data_clean2 = data_clean[['Close', 'High', 'Low', 'Month', 'Day', 'Hour', 'Minute', 'Volume',
       'RSI', '15_Volume', '15_ATR', '15_MACD_0', '15_MACD_1', '15_RSI',
       '15_StdDev', '15_stoc_0', '15_stoc_1', '15_Buying Climax',
       '15_Climactic Bar Strong', '15_Climactic Bar Weak',
       '15_HV Effort To Fall', '15_HV Effort To Rise', '15_ND SO',
       '15_NSP RSO', '15_Pseudo Reverse Upthrust', '15_Pseudo Upthrust',
       '15_Reverse Shake Out', '15_Reverse Upthrust', '15_Selling Climax',
       '15_Shake Out', '15_Stopping Volume', '15_Supply Test',
       '15_Upthrust', '15_Smart Buying', '15_Smart Selling',
       '60_Volume', '60_ATR', '60_MACD_0', '60_MACD_1', '60_RSI',
       '60_StdDev', '60_stoc_0', '60_stoc_1', '60_Buying Climax',
       '60_Climactic Bar Strong', '60_Climactic Bar Weak',
       '60_HV Effort To Fall', '60_HV Effort To Rise', '60_ND SO',
       '60_NSP RSO', '60_Pseudo Reverse Upthrust', '60_Pseudo Upthrust',
       '60_Reverse Shake Out', '60_Reverse Upthrust', '60_Selling Climax',
       '60_Shake Out', '60_Stopping Volume', '60_Supply Test',
       '60_Upthrust', '60_Smart Buying', '60_Smart Selling', 'Close_diff',
       'high_low_diff', 'MA_trend', '15_MA_trend', '15_MA_dist',
       '60_MA_dist', '15_MA50_dist', '60_MA50_dist', 'Trend', 'p_dist', 'r1_dist',
       'r2_dist', 'r3_dist', 's1_dist', 's2_dist', 's3_dist']]

In [ ]:
#USED FOR REINFORCEMENT LEARNING LSTM

data_clean2 = data_clean[['Close', 'High', 'Low', 'Month', 'Day', 'Hour', 'Minute', 'Volume',
       'RSI', '15_Volume', '15_ATR', '15_MACD_0', '15_MACD_1', '15_RSI',
       '15_StdDev', '15_stoc_0', '15_stoc_1', 'Close_diff',
       'high_low_diff', 'MA_trend', '15_MA_trend', '15_MA_dist',
       '60_MA_dist', '15_MA50_dist', '60_MA50_dist', 'Trend', 'p_dist', 'r1_dist',
       'r2_dist', 'r3_dist', 's1_dist', 's2_dist', 's3_dist',
       '60_Volume', '60_ATR', '60_MACD_0', '60_MACD_1', '60_RSI',
       '60_StdDev', '60_stoc_0', '60_stoc_1', '15_Buying Climax',
       '15_Climactic Bar Strong', '15_Climactic Bar Weak',
       '15_HV Effort To Fall', '15_HV Effort To Rise', '15_ND SO',
       '15_NSP RSO', '15_Pseudo Reverse Upthrust', '15_Pseudo Upthrust',
       '15_Reverse Shake Out', '15_Reverse Upthrust', '15_Selling Climax',
       '15_Shake Out', '15_Stopping Volume', '15_Supply Test',
       '15_Upthrust', '15_Smart Buying', '15_Smart Selling','60_Buying Climax',
       '60_Climactic Bar Strong', '60_Climactic Bar Weak',
       '60_HV Effort To Fall', '60_HV Effort To Rise', '60_ND SO',
       '60_NSP RSO', '60_Pseudo Reverse Upthrust', '60_Pseudo Upthrust',
       '60_Reverse Shake Out', '60_Reverse Upthrust', '60_Selling Climax',
       '60_Shake Out', '60_Stopping Volume', '60_Supply Test',
       '60_Upthrust', '60_Smart Buying', '60_Smart Selling']]

In [ ]:
#STACKING SAMPLES FOR XGBOOST - EXPERIMENTAL
from collections import deque
SEQ_LEN = 6

data_stack = np.empty([data_clean2.shape[0] - SEQ_LEN+1, SEQ_LEN*(data_clean2.shape[1]-10)])
data_np = np.array(data_clean2.iloc[:,3:-7])
chl = np.array(data_clean2.iloc[:,:3])
pivot_num = np.array(data_clean2.iloc[:,-7:])
data_deque = deque(maxlen=SEQ_LEN)

i = 0
for row in data_np:
    data_deque.append(row)
    if len(data_deque) == SEQ_LEN:
        row_stack = np.stack(data_deque, axis=1).reshape(1,-1)
        #row_stack = np.concatenate((chl[i:i+1], row_stack), axis=1)
        #row_stack = np.concatenate((row_stack, pivot_num[i:i+1]), axis=1)
        data_stack[i] = row_stack
        i += 1
stack = np.hstack((chl[SEQ_LEN-1:,:], data_stack, pivot_num[SEQ_LEN-1:,:]))

In [ ]:
#### USED TO TEST FOR NAN VALUES IF ALL TIMEFRAMES AREN'T ALIGNED 100% BY START/END TIME
testPD = data_clean2
null_columns=testPD.columns[testPD.isnull().any()]

testPD[null_columns].isnull().sum()
print(testPD[testPD.isnull().any(axis=1)][null_columns].head())

EXPORT

In [ ]:
export_csv = data_clean2.to_csv(f'data\\{SYMBOL}_2015_2020_Smart_Diff_RL_V1b.csv', index=False, header=None)

CREATE TARGET VALUE (Y) FOR DATASET FOR USE IN ORDINARY DNN OR XGBOOST

In [ ]:
#CALCULATE Y AS A TAKE PROFIT VALUE. IF POSITIVE VALUE IT SHOULD BUY AND HOLD UNTIL VALUE. IF NEGATIVE VALUE IT SHOULD SELL AND HOLD UNTIL VALUE REACHED.

y = [] 
counter = [0,0,0,0]
index = 0
for row in stack:
    if index > stack.shape[0]-30:
        print(index)
        break
    high = 0
    low = 0
    price = row[0]
    for j in range(20):
        if high < stack[index+j+1][1] - price: #High
            high = stack[index+j+1][1] - price
        if low > stack[index+j+1][2] - price: #Low
            low = stack[index+j+1][2] - price
    if high > abs(low):
        y.append(high)
        #y[i,1] = high
        counter[0] += 1
        counter[1] += high
    else:
        y.append(low)
        #y[i,1] = low
        counter[2] += 1
        counter[3] += low
    index += 1
            
print(f'DONE! High: {counter[0]}  Low: {counter[2]}')

In [ ]:
#CALCULATE Y AS CATEGORICAL VALUE: 0 - 1 - 2 (Should the model do nothing (0) - BUY (1) - SELL (2))
y = np.empty((data_clean2.shape[0],2))
counter = [0,0,0]
for index, row in data_clean2.iterrows():
    if index > data_clean2.shape[0]-30:
        break
    y[index,0] = index
    price = row.iloc[0]
    for i in range(18):
        if data_clean2.iloc[index+i+1][1] > price + (20/SYMBOL_FACTOR): #HIGH
            y[index,1] = 1
            counter[1] += 1
            break
        elif data_clean2.iloc[index+i+1][2] < price-(20/SYMBOL_FACTOR): #LOW
            y[index,1] = 2
            counter[2] += 1
            break
        else:
            if i == 11:
                y[index,1] = 0
                counter[0] += 1
            
print(f'DONE! Zero: {counter[0]}  Ones: {counter[1]}  Twos: {counter[2]}')

In [ ]:
# CLASSIFICATION
#ADD Y VALUES TO DATASET
data_adj = data_clean2.iloc[:-30,3:]
data_adj['y'] = pd.DataFrame(y[:-30,1:])
data_adj = data_adj.iloc[:-1,:]

In [ ]:
data_adj = np.concatenate((stack[:-29,3:], np.expand_dims(np.array(y), axis=1)), axis=1)

IF USING DATASET FOR XGBOOST WE NEED TO REMOVE OUTLIERS. A 4 STD. DIVIATION IS FOUND TO BE APPROPRIATE

In [ ]:
#PANDAS
#data_no_outliers = data_adj[np.abs(data_adj.y-data_adj.y.mean()) <= (4*data_adj.y.std())]

#NUMPY
data_no_outliers = data_adj[np.abs(data_adj[:,-1] - np.mean(data_adj[:,-1])) <= 4*np.std(data_adj[:,-1])]
# keep only the ones that are within +/-4 standard deviations in the column 'y'.
data_no_outliers.shape

In [ ]:
np_dt = data_no_outliers['y']
y_count = np.fabs(np_dt)
sum(y_count)/np_dt.shape[0]

In [ ]:
#Testing for NAN values
testPD = data_adj
null_columns=testPD.columns[testPD.isnull().any()]

testPD[null_columns].isnull().sum()
print(testPD[testPD.isnull().any(axis=1)][null_columns].head())

BALANCE AND SHUFFLE DATASET

BALANCE VALUE SET

In [ ]:
long = []
short = []

for index, row in data_adj.iterrows():
   
    if row['y'] >= 0:
        long.append(row)
    else:
        short.append(row)

In [ ]:
min_df = min(len(long),len(short))

long_df_s = (pd.DataFrame(long).sample(frac=1).reset_index(drop=True)).iloc[:min_df,:]
short_df_s = (pd.DataFrame(short).sample(frac=1).reset_index(drop=True)).iloc[:min_df,:]

df = long_df_s
df = pd.concat([df, short_df_s])
df = df.sample(frac=1).reset_index(drop=True)

BALANCE CLASSIFICATION SET

In [ ]:
null_df = []
one_df = []
two_df = []
for index, row in data_adj.iterrows():
    if int(row['y']) == 0:
        null_df.append(row)
    elif int(row['y']) == 1:
        one_df.append(row)
    elif int(row['y']) == 2:
        two_df.append(row)

In [ ]:
min_df = min(len(null_df),len(one_df),len(two_df))

null_df_s = (pd.DataFrame(null_df).sample(frac=1).reset_index(drop=True)).iloc[:min_df,:]
one_df_s = (pd.DataFrame(one_df).sample(frac=1).reset_index(drop=True)).iloc[:min_df,:]
two_df_s = (pd.DataFrame(two_df).sample(frac=1).reset_index(drop=True)).iloc[:min_df,:]
df = null_df_s
df = pd.concat([df, one_df_s])
df = pd.concat([df, two_df_s])
df = df.sample(frac=1).reset_index(drop=True)

EXPORT AS CSV OR NPY FILE

In [ ]:
data_np = data_clean2.to_numpy()

In [ ]:
np.save(f"data\\{SYMBOL}_2019_2020_Smart_Diff_RL_V2", data_np)

In [ ]:
with open(f"data\\WORK_Value_sample20_{SYMBOL}_2015_2019_Smart_Diff_Stack_V1.csv", "a") as f:
    np.savetxt(f, data_adj, fmt="%s", newline=' ', delimiter=',')
    f.write("\n")

In [ ]:
#testing for NAN values
data_adj.isnull().any() 

In [ ]:
export_csv = data_adj.to_csv(f'data\\WORK_Classif_sample18_{SYMBOL}_2015_2019_Smart_Diff_RLV1.csv')